# 1.0.0: Make a `Scene`!

## The `Scene` object

The `Scene` object is the primary object we'll be dealing with when working with `AudibleLight`. Our `Scene` object is comparable to `scaper.core.Scaper` or `spatialscaper.core.Scaper`, but with numerous adjustments that make data generation more straightforward and scalable.

A basic `Scene` object can be initialised as follows:

In [2]:
from audiblelight import utils
from audiblelight.core import Scene

In [3]:
scene = Scene(
    duration=60,
    mesh_path=utils.get_project_root() / "tests/test_resources/meshes/Oyens.glb"
)

CreateContext: Context created


`Scene.__init__` takes many optional arguments, which are described in more detail within the documentation.

## Using distributions

When a `Scene` is initialised, various distributions can be passed to allow for randomly sampling parameters such as event start times and durations.

These durations must satisfy the following conditions:
- Must be callable without arguments
- OR define an `rvs` method that is callable without arguments
- Must return a floating point value when called

This means that (for example), we can use `scipy` distributions, custom functions, etc.

In [ ]:
import numpy as np
import scipy.stats as stats


def truncated_gaussian():
    return np.clip(np.random.normal(5., 1.), 4, 6)


# All of these are valid distributions
scene = Scene(
    duration=60,
    mesh_path=utils.get_project_root() / "tests/test_resources/meshes/Oyens.glb",
    scene_start_dist=truncated_gaussian,
    event_start_dist=lambda: np.random.uniform(0.0, 10.0),
    event_velocity_dist=stats.uniform(10, 10)
)

When an `Event` is added with `Scene.add_event`, the following logic is used to decide whether the distributions passed to `Scene.__init__` should be sampled from:

- If overrides are passed directly to `add_event`, these will **always** be used
- If overrides are not passed but a valid distribution has been, this will be sampled
- If neither overrides nor a distribution has been passed, the value will be sampled from a sensible default distribution.

## Passing audio directories

We can pass `fg_path` to `Scene.__init__`. This allows us to define a directory (or list of directories) containing foreground audio. When we add an event with `Scene.add_event` **without also specifying** `filepath=...`, we'll pull in from this directory.

In [6]:
scene = Scene(
    duration=60,
    mesh_path=utils.get_project_root() / "tests/test_resources/meshes/Oyens.glb",
    fg_path=utils.get_project_root() / "tests/test_resources/soundevents/music"
)
scene.add_event(event_type="static", alias="will_be_music")
music_event = scene.get_event("will_be_music")
print(music_event.filepath)

CreateContext: Context created


CreateContext: Context created


2025-10-07 15:23:37.526 | INFO     | audiblelight.core:add_event:830 - Event added successfully: Static 'Event' with alias 'will_be_music', audio file '/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/music/007527.mp3' (unloaded, 0 augmentations), 1 emitter(s).


/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/music/007527.mp3


In this example, we pass in a directory containing music objects. When we add an `Event` to the `Scene`, we'll draw from this dictionary.

### Controlling duplicate audio files

By default, we allow a single unique audio file to appear numerous times in a `Scene`. In practice, this is usually not a problem as we would expect `fg_dir` to contain many audio files, and therefore duplicates (especially overlapping duplicates) are in reality very rare.

If this behaviour is undesirable, the argument `allow_duplicate_audios=False` can be passed when initialising a `Scene`:

In [10]:
no_dupes_allowed = Scene(
    duration=60,
    mesh_path=utils.get_project_root() / "tests/test_resources/meshes/Oyens.glb",
    fg_path=utils.get_project_root() / "tests/test_resources/soundevents/music",
    allow_duplicate_audios=False
)

# Add in some music files
for _ in range(2):
    no_dupes_allowed.add_event(event_type="static")

# Print the filepaths
events = no_dupes_allowed.get_events()
for ev in events:
    print(ev.filename)

CreateContext: Context created


2025-10-07 15:27:57.924 | INFO     | audiblelight.core:add_event:830 - Event added successfully: Static 'Event' with alias 'event000', audio file '/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/music/007527.mp3' (unloaded, 0 augmentations), 1 emitter(s).


CreateContext: Context created


2025-10-07 15:27:58.251 | INFO     | audiblelight.core:add_event:830 - Event added successfully: Static 'Event' with alias 'event001', audio file '/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/music/001666.mp3' (unloaded, 0 augmentations), 1 emitter(s).


CreateContext: Context created
007527.mp3
001666.mp3
